# はじめる

このノートブックを実行する前に、次のことを確認してください。

1. **ローカルモジュール：** 必要なローカルPythonファイル（`items.py`、` loaders.py`、`test.py`）をColab Runtimeの一時ストレージにアップロードする。これを行うには、左サイドバーのフォルダー・アイコン、次にアップロード・アイコンをクリックして、ファイルを選択。
2. 左側のサイドバーのキーアイコンをクリックし、[新しいシークレット]をクリックし、「hf_token」という名前のトークンを追加。
3. これらの手順が完了したら、残りのノートブックセルを順番に実行できます。

In [ ]:
# コースの環境に合わせて、ローカル環境から正確なバージョンをインストール
!pip install --upgrade pip

# 必要なライブラリの特定のバージョンをインストール
!pip install datasets==3.6.0
!pip install transformers==4.51.3
!pip install huggingface_hub==0.31.2
!pip install matplotlib==3.10.3
!pip install numpy==1.26.4
!pip install python-dotenv==1.1.0
!pip install tqdm==4.67.1

In [6]:
# import

import os
import random
import pickle

from dotenv import load_dotenv
from huggingface_hub import login

import numpy as np
from collections import Counter, defaultdict
from datasets import load_dataset, Dataset, DatasetDict
import matplotlib.pyplot as plt

In [7]:
# ColabのシークレットからHugging Faceアクセストークンを取得
# このトークンは、Hugging Face Hubと対話するために必要
from google.colab import userdata
# userdata.get('HF_TOKEN') # 危ない（笑）

In [8]:
# ローカルファイルからカスタムクラスをインポート（items.pyおよびloaders.py）
# これらのファイルは（、前述の通り）、Colab Runtimeの一時ストレージに手動で追加する必要がある。
from loaders import ItemLoader
from items import Item

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

In [9]:
# Matplotlibのバックエンドを設定して、ノートブックにインラインでプロットを表示
%matplotlib inline

## アイテムローダーコード

`loaders.py`を見る - 私たちの生活を楽にするための便利なコードがいくつかあります

```python
# 前回と同じデータセットをロード
items = ItemLoader("Home_and_Kitchen").load()
```
```text
Loading dataset Home_and_Kitchen
README.md: 0.00B [00:00, ?B/s]
Amazon-Reviews-2023.py: 0.00B [00:00, ?B/s]
meta_Home_and_Kitchen.jsonl:   0%|          | 0.00/11.8G [00:00<?, ?B/s]
Generating full split: 0 examples [00:00, ? examples/s]
Loading dataset shards:   0%|          | 0/21 [00:00<?, ?it/s]
100%|██████████| 3736/3736 [4:12:03<00:00,  4.05s/it]
Completed Home_and_Kitchen with 1,112,910 datapoints in 278.3 mins
```

```python
# アイテムを探す
print(items[1].prompt)
```
```text
How much does this cost to the nearest dollar?

Foaming Soap Dispenser Thick Ceramic Foam Hand Soap Dispenser for Bathroom or Kitchen Sink, Liquid Pump Bottles for Hand soap, Body Wash, 2 Pack Black
Saving money You can DIY foam soap which will save you hundreds of dollars ( create foam just need to dilute the regular soap with water in a ratio of 1 3 ). One pump foams the right amount of soap for a thorough hand wash. High Quality made of thickened ceramic material, which is sturdy, durable and wear-resistant. The base has a nice heavy weight so it won’t slide or tip when used. Simple use with 3.15 inch Wide opening and large pump button, you can refill and press very easily, even so easy for kids to use. Moreover, smoothly surface make it easy to clean

Price is $25.00
```

## 今、スケールアップします

大規模なホーム小売店で見つけるかもしれないすべてのアイテムのすべてのデータセットを見てみましょう - 電気、電子、オフィス関連で、衣服、美容、書籍ではありません。

1つのデータセットに４時間以上 ... Google Colabのセッション時間内に完了しないのでデータセット毎にpickleファイルに永続化して、最後に統合することにした。

In [13]:
# Colab Runtime環境で利用可能なCPUコアとRAMを確認して印刷
# これはデータ処理に使用されるリソースを理解するのに役立つ
import psutil
print(f"CPU cores: {psutil.cpu_count()}")
print(f"Available RAM: {psutil.virtual_memory().available / (1024**3):.1f} GB")

CPU cores: 2
Available RAM: 8.7 GB


In [ ]:
# Googleドライブをマウントして、ドライブ内のファイルにアクセス
from google.colab import drive
drive.mount('/content/drive')

In [12]:
# ロードされるデータセット名（Amazon製品カテゴリ）のリストを定義
dataset_names = [
    "Home_and_Kitchen",
    "Automotive",
    "Electronics",
    "Office_Products",
    "Tools_and_Home_Improvement",
    "Cell_Phones_and_Accessories",
    "Toys_and_Games",
    "Appliances",
    "Musical_Instruments",
]

'\n「自動車」「電化製品」\n「オフィス用品」\n「工具・住宅リフォーム」\n「携帯電話・アクセサリー」\n「玩具・ゲーム」\n「家電」「楽器」\n'

In [ ]:
# ファイル操作
import shutil

source_path = '/content/'
destination_path = '/content/drive/My Drive/'

items = []
for dataset_name in dataset_names:
    
    pkl_file = f'items_{dataset_name}.pkl'    
    if os.path.exists(pkl_file):
        print(f"{pkl_file} は既に存在するのでスキップします")
        continue
        
    loader = ItemLoader(dataset_name)
    #items.extend(loader.load())
    loaded_items = loader.load()
    
    # 各データセット毎に別々のpickleファイルに保存
    with open(pkl_file, 'wb') as f:
        pickle.dump(loaded_items, f)
    
    # ファイルをコピー
    shutil.copyfile(source_path + pkl_file, destination_path + pkl_file)
    # 確認メッセージをPrint
    print(f"Copied {source_path + pkl_file} to {destination_path + pkl_file}")
        
# さて、コーヒーブレイクの時間!!
# ちなみに、私は最大のデータセットを最初に置きます。それはより速くなります。

Loading dataset Automotive


meta_Automotive.jsonl:   0%|          | 0.00/5.35G [00:00<?, ?B/s]

In [ ]:
# 最後に統合
items = []
for dataset_name in dataset_names:
    pkl_file = f'{dataset_name}.pkl'
    with open(pkl_file, 'rb') as f:
        loaded_items = pickle.load(f)
        items.extend(loaded_items)

In [ ]:
# すべてのデータセットからロードされたアイテムの総数を印刷
print(f"A grand total of {len(items):,} items")

In [ ]:
# トークン数の分布を再度プロット
# MIN_TOKENS = 150 - MAX_TOKENS = 160 の範囲内なので
tokens = [item.token_count for item in items]
plt.figure(figsize=(15, 6))
plt.title(f"Token counts: Avg {sum(tokens)/len(tokens):,.1f} and highest {max(tokens):,}\n")
plt.xlabel('Length (tokens)')
plt.ylabel('Count')
plt.hist(tokens, rwidth=0.7, color="skyblue", bins=range(0, 300, 10))
plt.show()

In [ ]:
# 価格の分布を再度プロット

prices = [item.price for item in items]
plt.figure(figsize=(15, 6))
plt.title(f"Prices: Avg {sum(prices)/len(prices):,.1f} and highest {max(prices):,}\n")
plt.xlabel('Price ($)')
plt.ylabel('Count')
plt.hist(prices, rwidth=0.7, color="blueviolet", bins=range(0, 1000, 10))
plt.show()

In [ ]:
#「アイテムのカテゴリ毎の数」を棒グラフ（バーチャート）で可視化

# カウント集計
category_counts = Counter()
for item in items:
    category_counts[item.category]+=1

# 集計したカテゴリ名 (categories) と、その個数 (counts) のリストを作成
categories = category_counts.keys()
counts = [category_counts[category] for category in categories]

# カテゴリ別のバーチャート
plt.figure(figsize=(15, 6))
plt.bar(categories, counts, color="goldenrod")
plt.title('How many in each category')
plt.xlabel('Categories')
plt.ylabel('Count')

plt.xticks(rotation=30, ha='right')

# 各バーの上に値ラベルを追加します
for i, v in enumerate(counts):
    plt.text(i, v, f"{v:,}", ha='center', va='bottom')

# チャートを表示します
plt.show()

# 客観的

価格のバランスが取れているデータセットを作成します。安価なアイテムにはあまり重くなりませんでしたが、平均は60ドルを超えています。カテゴリのバランスを取るようにしてください - 自動車アイテムは少なくなります。

In [ ]:
# 価値でスロット分けした商品リストを作成（slots は list を返す dict）

slots = defaultdict(list) # キーが空の場合、空の list ([]) を自動で生成する dict
for item in items:
    slots[round(item.price)].append(item)

In [ ]:
# 価格の範囲ごと（slots）にアイテムを分け、サンプル・データセット（sample）を構築する
# ただし、自動車カテゴリ以外のアイテムに多くの重みを与えて抽出し、価格帯毎に偏りを抑える。

# 乱数シードの固定
np.random.seed(42)
random.seed(42)

# サンプルリスト生成
sample = []
for i in range(1, 1000): # 価格帯ごと（1-999$）
    slot = slots[i] # 当該価格の商品数
    if i>=240:              # 240＄以上なら全件追加（数が少ないので）
        sample.extend(slot)
    elif len(slot) <= 1200: # 1200件以下なら全件追加
        sample.extend(slot)
    else:                   # 上記以外、重みをAutomotiveは1、他は5にして1200件サンプリング
        weights = np.array([1 if item.category=='Automotive' else 5 for item in slot])
        weights = weights / np.sum(weights)
        selected_indices = np.random.choice(len(slot), size=1200, replace=False, p=weights)
        selected = [slot[i] for i in selected_indices]
        sample.extend(selected)

print(f"There are {len(sample):,} items in the sample")

In [ ]:
# サンプルの価格の分布をプロット
# ・200＄位から1,200件を下回る。
# ・X99＄のスロットは商品数は増える。
prices = [float(item.price) for item in sample]
plt.figure(figsize=(15, 10))
plt.title(f"Avg {sum(prices)/len(prices):.2f} and highest {max(prices):,.2f}\n")
plt.xlabel('Price ($)')
plt.ylabel('Count')
plt.hist(prices, rwidth=0.7, color="darkblue", bins=range(0, 1000, 10))
plt.show()

In [ ]:
# 「各カテゴリにいくつのアイテムがあるか」を分かりやすいバーチャートにして、各カテゴリの上に件数も明示

# カテゴリごとの件数を数える
category_counts = Counter()
for item in sample:
    category_counts[item.category]+=1

# カテゴリ名と件数のリスト作成
categories = category_counts.keys()
counts = [category_counts[category] for category in categories]

# バーチャートを作成
plt.figure(figsize=(15, 6))
plt.bar(categories, counts, color="lightgreen")

# チャートをカスタマイズ
plt.title('How many in each category')
plt.xlabel('Categories')
plt.ylabel('Count')

plt.xticks(rotation=30, ha='right')

# 各バーの上に値ラベルを追加
for i, v in enumerate(counts):
    plt.text(i, v, f"{v:,}", ha='center', va='bottom')

# チャートを表示
plt.show()

In [ ]:
# 別の視点については「ドーナツチャート（円グラフ）を見てみましょう

plt.figure(figsize=(12, 10))
plt.pie(counts, labels=categories, autopct='%1.0f%%', startangle=90)

# 中央に円を追加してドーナツチャートを作成（オプション）
centre_circle = plt.Circle((0,0), 0.70, fc='white')
fig = plt.gcf()
fig.gca().add_artist(centre_circle)
plt.title('Categories')

# 平等なアスペクト比は、パイが円として描かれることを保証
plt.axis('equal')

plt.show()

# データセットキュレーション！

優れたデータセットを作成しました。

いくつかの最終チェックをしましょう

In [ ]:
# プロンプト文字数と価格の関係は？

# 文字数と価格の間にうっすら相関関係を確認できる。
sizes = [len(item.prompt) for item in sample]
prices = [item.price for item in sample]

# 散布図を作成
plt.figure(figsize=(15, 8))
plt.scatter(sizes, prices, s=0.2, color="red")

# ラベルとタイトルを追加
plt.xlabel('Size')
plt.ylabel('Price')
plt.title('Is there a simple correlation?')

# プロットを表示
plt.show()

In [ ]:
# プロンプトと、下10トークンの確認
def report(item):
    prompt = item.prompt
    tokens = Item.tokenizer.encode(item.prompt)
    print(prompt)
    print(tokens[-10:])
    print(Item.tokenizer.batch_decode(tokens[-10:]))

In [ ]:
report(sample[398000])

## 観察

Llamaトークナイザーの興味深い点は、GPT-4oで見たように、1〜999のすべての数値が1トークンにマッピングされることです。同じことは、すべての数字をトークンにマッピングするQWEN2、GEMMA、およびPHI3にも当てはまりません。これは、私たちのプロジェクトにとって少し便利であることが判明しますが、重要な要件ではありません。

# ついに

データをトレーニング、テスト、検証データセットに分割する時が来ました。

テスト目的でデータの5％〜10％を使用することが典型的ですが、実際にはこの時点で必要なものよりもはるかに多くあります。トレーニングには400,000ポイントを獲得し、テスト用に2,000ポイントを予約しますが、すべてを使用することはありません。


In [ ]:
# データを train / test に分割
random.seed(42)
random.shuffle(sample)
train = sample[:400_000]
test = sample[400_000:402_000]
print(f"Divided into a training set of {len(train):,} items and test set of {len(test):,} items")

In [ ]:
# trainのプロンプト
print(train[0].prompt)

In [ ]:
# testのプロンプト
print(test[0].test_prompt())

In [ ]:
# 最初の250テストポイントで価格の分布をプロットする

prices = [float(item.price) for item in test[:250]]
plt.figure(figsize=(15, 6))
plt.title(f"Avg {sum(prices)/len(prices):.2f} and highest {max(prices):,.2f}\n")
plt.xlabel('Price ($)')
plt.ylabel('Count')
plt.hist(prices, rwidth=0.7, color="darkblue", bins=range(0, 1000, 10))
plt.show()

# 最後に - 新しいデータセットをアップロードします

プロンプトに変換し、Hugging Face Hubにアップロードします

In [ ]:
train_prompts = [item.prompt for item in train]
train_prices = [item.price for item in train]
test_prompts = [item.test_prompt() for item in test]
test_prices = [item.price for item in test]

In [ ]:
# listからdictを作成

train_dataset = Dataset.from_dict({"text": train_prompts, "price": train_prices})
test_dataset = Dataset.from_dict({"text": test_prompts, "price": test_prices})
dataset = DatasetDict({
    "train": train_dataset,
    "test": test_dataset
})

In [ ]:
# 実行前に、HFユーザー名に置き換え、Hubにプッシュする準備を完了させる。

hf_user = "nishi74322014"
DATASET_NAME = f"{hf_user}/pricer-data"
dataset.push_to_hub(DATASET_NAME, private=True)

アップロードを確認：https://huggingface.co/nishi74322014/datasets

In [ ]:
# train / test を pickle

with open('train.pkl', 'wb') as file:
    pickle.dump(train, file)

with open('test.pkl', 'wb') as file:
    pickle.dump(test, file)

In [ ]:
# Googleドライブをマウントして、ドライブ内のファイルにアクセスします
from google.colab import drive
drive.mount('/content/drive')

Googleドライブがマウントされたら、ドライブのフォルダーにファイルをコピーできます。 「私のドライブ/your_folder_name」を、ファイルを保存するフォルダーへのパスに置き換えます。

In [ ]:
# ファイル操作のためにShotilモジュールをインポートします
import shutil

# Googleドライブの宛先パスと、ピクルストレーニングデータのソースパスを定義します
# Googleドライブの目的のフォルダーパスに「My Drive/Your_folder_name」を交換してください
destination_path = '/content/drive/My Drive/train.pkl'
source_path = '/content/train.pkl'

# Colab環境からGoogleドライブに漬けられたトレーニングデータファイルをコピーする
shutil.copyfile(source_path, destination_path)

# 確認メッセージを印刷します
print(f"Copied {source_path} to {destination_path}")

In [ ]:
# ファイル操作のためにShotilモジュールをインポートします
import shutil

# Googleドライブの宛先パスとピクルステストデータのソースパスを定義する
# Googleドライブの目的のフォルダーパスに「My Drive/Your_folder_name」を交換してください
destination_path = '/content/drive/My Drive/test.pkl'
source_path = '/content/test.pkl'

# Colab環境からGoogleドライブに漬物のテストデータファイルをコピーする
shutil.copyfile(source_path, destination_path)

# 確認メッセージを印刷します
print(f"Copied {source_path} to {destination_path}")